In [1]:
import sys, os, itertools
import re, nltk
from nltk import pos_tag
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import sklearn, gensim
from sklearn.decomposition import PCA
from gensim.corpora import Dictionary
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
os.chdir(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/data/')
from utils import preprocess_for_bow
from models.lda import LDAwrappers
from models.hdp import HDPwrapper
from models.gsdmm import MovieGroupProcessWrapper
from models.lftm import LFTMwrapper
from models.bertopic import BERTopicWrapper
from sklearn.feature_extraction.text import CountVectorizer
from scipy.cluster import hierarchy as sch

In [2]:
data = preprocess_for_bow('data.csv', preprocessing=False)

### KeyBERTInspired

In [18]:
from bertopic.representation import KeyBERTInspired
representation_model = KeyBERTInspired()
bertopic=BERTopicWrapper(data['text'], data['ids'])
bertopic.model.fit(data['text'])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [19]:
bertopic.model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,12636,-1_is_of_the_covid,"[is, of, the, covid, for, was, are, covid19, i...",[All these governors are cooperatively working...
1,0,592,0_masks_mask_face_wearing,"[masks, mask, face, wearing, wear, spread, ine...",[WHO now saying You do not need to Wear a Mask...
2,1,550,1_tweet_contribute_menu_helpful,"[tweet, contribute, menu, helpful, might, misl...",[If you want to add helpful context to any Twe...
3,2,546,2_pope_church_catholic_bishops,"[pope, church, catholic, bishops, cardinal, fr...","[He said the pope is…, Pope Francis was arrest..."
4,3,359,3_de_en_la_un,"[de, en, la, un, el, que, del, foto, con, da]",[¡Hoy le cumplimos a Colombia! Desde el @BOG_E...
...,...,...,...,...,...
671,670,6,670_south_cryil_fifthhighest_unequivocally,"[south, cryil, fifthhighest, unequivocally, su...",[South Africa's president asks foreigners to l...
672,671,6,671_second_impeachment_patriotically_plagiarized,"[second, impeachment, patriotically, plagiariz...",[Former U.S. President Donald Trump said that ...
673,672,6,672_genital_mutilation_rand_decides,"[genital, mutilation, rand, decides, chelsea, ...",[Chelsea & the NY Times decides to sugar coat ...
674,673,6,673_fruits_vegetables_spread_shadab,"[fruits, vegetables, spread, shadab, tomato, v...",[Man used injection filled with spit on fruits...


In [20]:
bertopic.model.update_topics(data['text'], representation_model=representation_model)

In [21]:
bertopic.model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,12636,-1_coronavirus_covid_vaccine_pandemic,"[coronavirus, covid, vaccine, pandemic, covid1...",[All these governors are cooperatively working...
1,0,592,0_masks_mask_maskwearing_masking,"[masks, mask, maskwearing, masking, maskless, ...",[WHO now saying You do not need to Wear a Mask...
2,1,550,1_birdwatch_tweet_contributions_contribute,"[birdwatch, tweet, contributions, contribute, ...",[If you want to add helpful context to any Twe...
3,2,546,2_pope_popes_vatican_catholic,"[pope, popes, vatican, catholic, catholics, bi...","[He said the pope is…, Pope Francis was arrest..."
4,3,359,3_estado_confinamento_como_una,"[estado, confinamento, como, una, colombia, es...",[¡Hoy le cumplimos a Colombia! Desde el @BOG_E...
...,...,...,...,...,...
671,670,6,670_covid19_africa_africas_african,"[covid19, africa, africas, african, africans, ...",[South Africa's president asks foreigners to l...
672,671,6,671_impeachmenttrial_impeachment_impeached_tes...,"[impeachmenttrial, impeachment, impeached, tes...",[Former U.S. President Donald Trump said that ...
673,672,6,672_mutilation_genital_transgender_genitals,"[mutilation, genital, transgender, genitals, c...",[Chelsea & the NY Times decides to sugar coat ...
674,673,6,673_coronavirus_virus_fruits_vegetables,"[coronavirus, virus, fruits, vegetables, tomat...",[Man used injection filled with spit on fruits...


### MaximalMarginalRelevance

In [11]:
from bertopic.representation import MaximalMarginalRelevance
representation_model = MaximalMarginalRelevance(diversity=0.3)
bertopic=BERTopicWrapper(data['text'], data['ids'])
bertopic.model.fit(data['text'])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [12]:
bertopic.model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,12313,-1_of_for_the_is,"[of, for, the, is, in, was, covid19, trump, ar...",[This has been my primary problem talking abou...
1,0,597,0_masks_mask_face_wearing,"[masks, mask, face, wearing, wear, ineffective...","[Wearing face masks is harmful for health., Fa..."
2,1,549,1_tweet_contribute_menu_helpful,"[tweet, contribute, menu, helpful, might, misl...",[If you want to add helpful context to any Twe...
3,2,543,2_pope_church_catholic_bishops,"[pope, church, catholic, bishops, cardinal, fr...","[Not pope., Does Mr. Ferrara believe that the ..."
4,3,452,3_hoax_covid_deep_nwo,"[hoax, covid, deep, nwo, sheeple, world, fake,...",[The Great Covid Hoax will be another one. We ...
...,...,...,...,...,...
676,675,6,675_colorado_voter_kemp_georgia,"[colorado, voter, kemp, georgia, comparison, m...",[@SenatorTimScott @MLB Colorado mails every re...
677,676,6,676_pakistan_naval_plane_karachi,"[pakistan, naval, plane, karachi, pilot, allah...",[International Civil Aviation Organisation bar...
678,677,6,677_karachi_plane_pia_pakistan,"[karachi, plane, pia, pakistan, crash, interio...",[Video shows interiors of plane that crashed i...
679,678,6,678_testing_lizer_slowdown_myron,"[testing, lizer, slowdown, myron, navajo, sarc...",[He was being sarcastic when he said he had or...


In [13]:
bertopic.model.update_topics(data['text'], representation_model=representation_model)

In [14]:
bertopic.model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,12313,-1_for_covid19_trump_an,"[for, covid19, trump, an, not, vaccine, us, vi...",[This has been my primary problem talking abou...
1,0,597,0_masks_mask_wear_vaccinated,"[masks, mask, wear, vaccinated, prevent, cdc, ...","[Wearing face masks is harmful for health., Fa..."
2,1,549,1_tweet_contribute_menu_helpful,"[tweet, contribute, menu, helpful, might, bird...",[If you want to add helpful context to any Twe...
3,2,543,2_pope_bishops_cardinal_priests,"[pope, bishops, cardinal, priests, communion, ...","[Not pope., Does Mr. Ferrara believe that the ..."
4,3,452,3_hoax_covid_nwo_reset,"[hoax, covid, nwo, reset, agenda, control, con...",[The Great Covid Hoax will be another one. We ...
...,...,...,...,...,...
676,675,6,675_colorado_kemp_comparison_kemps,"[colorado, kemp, comparison, kemps, wokes, mlb...",[@SenatorTimScott @MLB Colorado mails every re...
677,676,6,676_naval_karachi_allahs_licences,"[naval, karachi, allahs, licences, sneaks, pns...",[International Civil Aviation Organisation bar...
678,677,6,677_karachi_plane_pia_crash,"[karachi, plane, pia, crash, footage, horrific...",[Video shows interiors of plane that crashed i...
679,678,6,678_testing_lizer_slowdown_myron,"[testing, lizer, slowdown, myron, navajo, volu...",[He was being sarcastic when he said he had or...


### POS

## Generative LLM

In [8]:
from transformers import pipeline
from bertopic.representation import TextGeneration

prompt = "I have a topic described by the following keywords: [earth, CO2, gaz]. Based on the previous keywords, what is this topic about?"

generator = pipeline('text2text-generation', model='google/flan-t5-base')
representation_model = TextGeneration(generator, "I have a topic described by the following keywords: [KEYWORDS]. Based on the previous keywords, what is this topic about?")
bertopic=BERTopicWrapper(data['text'], data['ids'])
bertopic.model.fit(data['text'],)
bertopic.model.get_topic_info()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

,Topic,Count,Name,Representation,Representative_Docs
0,-1,12525,-1_the_is_of_trump,"[the, is, of, trump, for, in, covid19, covid, ...",[I heard a healthcare employee I’m friends wit...
1,0,595,0_masks_mask_face_wearing,"[masks, mask, face, wearing, wear, ineffective...","[Wearing masks won’t prevent coronavirus, Face..."
2,1,551,1_tweet_contribute_menu_helpful,"[tweet, contribute, menu, helpful, might, misl...",[If you want to add helpful context to any Twe...
3,2,549,2_pope_church_catholic_bishops,"[pope, church, catholic, bishops, francis, car...","[He said the pope is…, Does Mr. Ferrara believ..."
4,3,351,3_de_en_la_un,"[de, en, la, un, el, que, del, foto, da, una]",[¡Hoy le cumplimos a Colombia! Desde el @BOG_E...
...,...,...,...,...,...
686,685,6,685_bengal_west_dates_storming,"[bengal, west, dates, storming, elections, boo...",[West Bengal Election Dates Have Been Announce...
687,686,6,686_christian_christianity_recant_duane,"[christian, christianity, recant, duane, waver...",[I’m going on record now: If they try to cance...
688,687,6,687_formaldehyde_ingredients_toxic_placebos,"[formaldehyde, ingredients, toxic, placebos, l...",[Formaldehyde in vaccines is connected to chil...
689,688,6,688_rafale_farewell_jets_tricolour,"[rafale, farewell, jets, tricolour, france, da...",[Viral Video of France's Farewell to Rafale wi...


In [9]:
bertopic.model.update_topics(data['text'], representation_model=representation_model)
bertopic.model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,12525,-1_trump___,"[trump, , , , , , , , , ]",[I heard a healthcare employee I’m friends wit...
1,0,595,0_Science/Tech___,"[Science/Tech, , , , , , , , , ]","[Wearing masks won’t prevent coronavirus, Face..."
2,1,551,1_birdwatch___,"[birdwatch, , , , , , , , , ]",[If you want to add helpful context to any Twe...
3,2,549,2_catholic church___,"[catholic church, , , , , , , , , ]","[He said the pope is…, Does Mr. Ferrara believ..."
4,3,351,3_colombia___,"[colombia, , , , , , , , , ]",[¡Hoy le cumplimos a Colombia! Desde el @BOG_E...
...,...,...,...,...,...
686,685,6,685_elections___,"[elections, , , , , , , , , ]",[West Bengal Election Dates Have Been Announce...
687,686,6,686_christianity___,"[christianity, , , , , , , , , ]",[I’m going on record now: If they try to cance...
688,687,6,687_World___,"[World, , , , , , , , , ]",[Formaldehyde in vaccines is connected to chil...
689,688,6,688_World___,"[World, , , , , , , , , ]",[Viral Video of France's Farewell to Rafale wi...


In [ ]:
keybert = KeyBERTInspired()
mmr = MaximalMarginalRelevance(diversity=0.3)
chained_representation = [keybert, mmr]